<a href="https://colab.research.google.com/github/ymuto0302/RW2024/blob/main/MLP_classification_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:
# DataSetの準備
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# データの生成（Irisデータセットを用いる）
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# データセットの作成
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)


In [ ]:
# DataLoader の設定
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# モデルの定義
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()

        # 入力層から最初の隠れ層
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.layers.append(nn.ReLU())

        # 隠れ層
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.layers.append(nn.ReLU())

        # 出力層
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# nn.Sequential を利用した実装でも OK。ただし，隠れ層の数は固定。
class MLP2(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP2, self).__init__()

        self.net = nn.Sequential(
           nn.Linear(input_size, hidden_sizes[0]),
           nn.ReLU(),
           nn.Linear(hidden_sizes[0], hidden_sizes[1]),
           nn.ReLU(),
           nn.Linear(hidden_sizes[1], output_size),
        )

    def forward(self, x):
        return self.net(x)

# モデルのインスタンス化
input_size = 4  # Irisデータセットの特徴量の数
hidden_sizes = [100, 50]  # 隠れ層のニューロン数
output_size = 3  # Irisデータセットのクラス数
model = MLP(input_size, hidden_sizes, output_size)


In [ ]:
# 損失化関数と最適化アルゴリズムの設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# モデルの学習
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        # フォワードパス
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # バックワードパスと最適化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # エポックごとの損失を表示
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.5674
Epoch [20/100], Loss: 0.3937
Epoch [30/100], Loss: 0.2646
Epoch [40/100], Loss: 0.1665
Epoch [50/100], Loss: 0.1839
Epoch [60/100], Loss: 0.0675
Epoch [70/100], Loss: 0.0736
Epoch [80/100], Loss: 0.0294
Epoch [90/100], Loss: 0.0403
Epoch [100/100], Loss: 0.1491


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    print(f'Accuracy on the test set: {100.0 * correct / total:.2f}%')

Accuracy on the test set: 96.67%
